In [1]:
import numpy as np
from math import *
from scipy import integrate
from cyfuncs import zhao_func
from exceptions import Exception, ValueError, OverflowError, ZeroDivisionError

In this notebook I integrate the DM squared density (to obtain the *J-factor*) using the same strategy as in CLUMPY:  
I split the integral into three regions:  
- the "front" part of the Halo, zone 1   
- the "rear" part of the Halo, zone 2  
- the inner cusp/core, zone 3  

In [2]:
# physical parameters as in Alex's simulations (except rt)
D = 40.
r0 = 0.35
rt = np.inf

theta = 0.5
Dprime = D/r0
rtprime = rt/r0
ymin = cos( np.radians(theta) )

def density(x):
    return zhao_func(x, 1., 3., 0.)

def radius(z, y, Dprime):
    try:
        return sqrt( z*z + Dprime*Dprime*(1-y*y) )
    except (OverflowError, ZeroDivisionError):
        return np.nan

These functions implement the generalised NFW (as in the module `cyfuncs.pyx`)

In [3]:
def integrand(z, y, ymin, rc):
    try:
        return density( radius(z, y, Dprime) )**2
    except (OverflowError, ZeroDivisionError):
        return np.nan

rc = 0.001

# l.o.s. integration-variable l boundaries for zone 1
def lim_l1(y, ymin, rc):
    alpha = atan( rc/D )
    A = cos( np.radians(alpha) )
    l1_min = D*y - sqrt( rt*rt - D*D* (1. - y*y) )
    l1_max = D*y if y<A else D*A - sqrt( rc*rc - D*D * (1. - A*A) )
    return [l1_min, l1_max]

# l.o.s. integration-variable l boundaries for zone 2
def lim_l2(y, ymin, rc):
    alpha = atan( rc/D )
    A = cos( np.radians(alpha) )
    l2_min = D*y if y<A else D*A + sqrt( rc*rc - D*D * (1. - A*A) )
    l2_max = D*y + sqrt( rt*rt - D*D * (1. - y*y) )
    return [l2_min, l2_max]

# l.o.s. integration-variable l boundaries for zone 3
def lim_l3(A, ymin, rc):
    l3_min = D*A - sqrt( rc*rc - D*D * (1. - A*A) )
    l3_max = D*A + sqrt( rc*rc - D*D * (1. - A*A) )
    return [l3_min, l3_max]

def lim_y(ymin, rc):
    return [ymin, 1.]

In [35]:
# here I evaluate the two integrals separately

I1 = integrate.nquad(integrand, ranges=[lim_l1, lim_y], args=(ymin, rc), \
                    opts=[{'limit':1000, 'epsabs':1.e-16, 'epsrel':1.e-16},\
                          {'limit':1000, 'epsabs':1.e-16, 'epsrel':1.e-16}])

I2 = integrate.nquad(integrand, ranges=[lim_l2, lim_y], args=(ymin, rc), \
                    opts=[{'limit':1000, 'epsabs':1.e-16, 'epsrel':1.e-16},\
                          {'limit':1000, 'epsabs':1.e-16, 'epsrel':1.e-16}])

I1,I2 

((3.1540608084866464e-06, 4.9939011011331466e-15),
 (6.569491689401453e-14, 2.9524398816015757e-19))

In [36]:
# here I evaluate the integral over the inner cusp/core region
Amin = cos( np.radians( atan( rc/D ) ) )
I3 = integrate.nquad(integrand, ranges=[lim_l3, lim_y], args=(Amin, rc), \
                    opts=[{'limit':1000, 'epsabs':1.e-16, 'epsrel':1.e-16},\
                          {'limit':1000, 'epsabs':1.e-16, 'epsrel':1.e-16}])

I3

(4.0057596480735347e-26, 4.6745198661020834e-27)

In [37]:
# finally, I compare the result of the NQUAD integration routine with ROMBERG method
def integrand_y(y, ymin, rc):
    return integrate.romberg( integrand, a=lim_l3(y, ymin, rc)[0], b=lim_l3(y, ymin, rc)[1], args=(y, ymin, rc), tol=1.e-16 )

Iromb = integrate.romberg( integrand_y, a=lim_y(Amin, rc)[0], b=lim_y(Amin, rc)[1], tol=1.e-10, args=(ymin, rc) )

Iromb

4.0057595294703211e-26

In [38]:
I1[0] + I2[0] + I3[0]

3.1540608741815634e-06